## selenium을 이용하여 주유가격 수집 2

**일일히 css에 스크레핑해와야 쥬피터를 돌렸을때 뜬다.**

In [77]:
from selenium import webdriver as wd
import urllib

In [78]:
driver = wd.Chrome('../tool/chromedriver_75.0.3770.140.exe')

In [79]:
target_url = 'http://www.opinet.co.kr/searRgSelect.do'

In [80]:
# 위 사이트가 중간 페이지를 직접 접속하는 부분을 거부하기 때문에 
# 시간차를 두고 2회 요청을 진행했다.
import time
for n in range(2):
    driver.get( target_url )
    time.sleep(5)
    print('요청중',n+1)

요청중 1
요청중 2


1. '시도' 정보를 추출하여 리스트화 
2. 리스트를 반복하면서 해당 '시도' 밑에 '시군구' 리스트화 
3. '시군구' 리스트를 반복하면서 '읍면동'도 리스트화
4. '읍면동' 리스트를 반복하면서 -> 조회 -> 다운로드순으로 진행되는것이 기본 
5. 사이트 관찰결과, '시도','시군구','읍면동' 정보가 새로 세팅되면서 화면의 DOM이 모두 새로 구성이 되는 문제가 있어서 반복문 안에서는 반드시 '시도','시군구','읍면동',을 새로 잡아줘야한다.!!(매우중요)
6. 여기서는'시군구'를 반복하면서 바로 조회 및 다운로드로 전개!!!

In [81]:
# 1. '시도' 정보를 추출하여 리스트화 

sidos = [ option.get_attribute('value') 
          for option in driver.find_elements_by_css_selector('#SIDO_NM0 > option') 
          if option.get_attribute('value') 
        ]
#데이터의 상위 값, 하위 값의 각 값을 조사를 통해 샘플링으로 확인! 
sidos[:2],sidos[-2:],sidos[1:2]

(['서울특별시', '부산광역시'], ['경상남도', '제주특별자치도'], ['부산광역시'])

In [82]:
# 2. 리스트를 반복하면서 해당 '시도' 밑에 '시군구' 리스트화 
#sidos 중 부산만 대상으로 수집!!
# 샘플링을 통해서 sidos[1:2]가 부산 광역시임을 확인_
for sido in sidos[1:2]:
    #2-1. sido의 대상을 find
    sido_tag = driver.find_element_by_id('SIDO_NM0')
    #2-2. sido 값을 설정
    sido_tag.send_keys( sido )
    #2-2-1.잠시 대기 5초(ajax 통신후 화면이 재구성될대까지)
    time.sleep(5)
    #2-3. '시구군' 리스트 확득
    sigungus = [option.get_attribute('value') 
                for option in driver.find_elements_by_css_selector('#SIGUNGU_NM0 > option') 
                if option.get_attribute('value') 
    ]
    print(sido, sigungus[:4])
    #2-4. 반복 ~ 3단계로 진입 
    
    #3. '시군구' 리스트를 반복하면서 -> 잠시 대기 (5sec) -> 엑셀로 저장
    for sigungu in sigungus:
        #3-1.' 시군구' 태그를 찾아서 
        sigungu_tag = driver.find_element_by_id('SIGUNGU_NM0')
        #3-2. ' 시군구' 값을 설정 
        sigungu_tag.send_keys( sigungu )
        #3-3. 잠시대기 
        time.sleep(5)
        #엑셀 저장 
        driver.find_element_by_id('glopopd_excel').click()
    

부산광역시 ['강서구', '금정구', '기장군', '남구']


## 데이터 수집 완료 
- 수십개의 엑셀파일을 반복적으로 읽어서 DataFrame으로 구성하기

In [83]:
# glob 특성 위치상에 존재하는 파일들을 패턴을 지정하여 목록을 구성할수 있다. 
from glob import glob
import pandas as pd

In [89]:
# 부산에 관한 '시군구'에 조회된 엑셀 자료를 읽을 수 있도록 파일경로를 리스트화 
down_files = glob('C:/Users/sun41/Desktop/projects/analysis/pandas/data/opinet/지역*.xls')
down_files[:2],down_files[-2:]

(['C:/Users/sun41/Desktop/projects/analysis/pandas/data/opinet\\지역_위치별(주유소) (1).xls',
  'C:/Users/sun41/Desktop/projects/analysis/pandas/data/opinet\\지역_위치별(주유소) (10).xls'],
 ['C:/Users/sun41/Desktop/projects/analysis/pandas/data/opinet\\지역_위치별(주유소) (9).xls',
  'C:/Users/sun41/Desktop/projects/analysis/pandas/data/opinet\\지역_위치별(주유소).xls'])

In [90]:
# down_files로 엑셀 파일의 경로와 이름을 리트스로 저장 -> read_excel로 각파일을 읽은후 
# tmp_raws 변수에 append 시킴
tmp_raws = []
for file in down_files:
    # read_excel로 각 엑셀파일을 읽은 후 
    tmp = pd.read_excel(file, header =2)
    #타겟을 정하고 내용을 삽입 (append) 
    tmp_raws.append(tmp )
#반복문이 끝나면concat으로 엑셀 내용이 하나로 합쳐진다.
st_raw = pd.concat(tmp_raws)

In [91]:
#스테이션 정보가 저장된것을 확인
st_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420 entries, 0 to 48
Data columns (total 10 columns):
지역       420 non-null object
상호       420 non-null object
주소       420 non-null object
상표       420 non-null object
전화번호     420 non-null object
셀프여부     420 non-null object
고급휘발유    420 non-null object
휘발유      420 non-null object
경유       420 non-null object
실내등유     420 non-null object
dtypes: object(10)
memory usage: 36.1+ KB


In [92]:
st_raw.shape

(420, 10)

In [95]:
st_raw.columns

Index(['지역', '상호', '주소', '상표', '전화번호', '셀프여부', '고급휘발유', '휘발유', '경유', '실내등유'], dtype='object')

In [101]:
st_raw.columns[2:3]

Index(['주소'], dtype='object')

In [94]:
#상위 3개 - 컬럼 제외
st_raw.head(3)

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,부산광역시,은마석유 노포주유소,부산 금정구 중앙대로 2191 (노포동),GS칼텍스,051-508-1645,Y,-,1463,1308,1050
1,부산광역시,훼미리알뜰주유소,부산 금정구 두실로 2 (남산동),알뜰주유소,051-582-5882,N,-,1473,1318,-
2,부산광역시,청룡명품주유소,부산 금정구 중앙대로 2130,현대오일뱅크,051-508-5150,N,-,1478,1318,-


In [102]:
st_raw.tail(3)

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
46,부산광역시,SK에너지(주)새부산주유소,부산 강서구 낙동북로 42 (강동동),SK에너지,051-971-8484,N,-,1559,1433,1020
47,부산광역시,명지주유소,부산 강서구 낙동남로 1042,SK에너지,051-271-2828,N,-,1568,1428,-
48,부산광역시,동성셀프주유소,부산광역시 강서구 명지국제8로 272 (명지동),S-OIL,051-711-7537,Y,-,1569,1409,-


- 응용 : '시군구' 단위 => '읍면동' 단위 

> 1. 주유소정보데이터 부산시 행정구역으로 표현  
> 2. 자치구별 가장 저렴한 주유소 및 가장 비싼 주유소 마킹  
> 3. 행정구역별 주유가격 평균을 통해서 칼라맵으로 표시, 휘발유기준, 가격없는 주유소는 배제  
> 4. 브랜드별(6개) 알뜰주유소 Vs 브랜드대비, 진짜 싼지 체크!!  -> seaborn의 박스플롯과 스완플롯을 같이 활용하여 시각화 한다
> 브랜드, 셀프여부, 휘발유/경유,등등

In [103]:
# 주소가 xx동이 탐색되는 있는 주유소 정보
tmp = st_raw[ st_raw['주소'].apply(lambda x: x.find('동')< 0)]
tmp.shape

(96, 10)

In [104]:
# 값 중복성 제거를 통안 결측치 및 타입 확인 
st_raw['휘발유'].unique()

array([1463, 1473, 1478, 1483, 1489, 1498, 1503, 1509, 1527, 1528, 1569,
       1579, 1585, 1617, 1695, 1459, 1475, 1488, 1495, 1518, 1598, 1482,
       1486, 1487, 1514, 1517, 1538, 1545, 1597, 1650, 1688, 1698, 1458,
       1468, 1492, 1496, 1510, 1513, 1515, 1519, 1558, 1557, 1578, 1658,
       1694, 1434, 1565, 1449, 1457, 1462, 1479, 1485, 1529, 1533, 1535,
       1548, 1550, 1553, 1555, 1568, 1583, 1593, 1633, 1499, 1508, 1540,
       1551, 1699, 1476, 1477, 1497, 1525, 1549, 1589, 1595, 1596, 1493,
       1885, 1466, 1507, 1725, '1445', '1448', '1458', '1459', '1468',
       '1475', '1477', '1486', '1488', '1495', '1497', '1527', '1528',
       '1548', '1555', '1568', '1595', '1598', '-', 1448, 1454, 1506,
       1559, 1572, 1443, 1444, 1445, 1556, 1629, 1637, 1739, '1440',
       '1447', '1467', '1478', '1487', '1493', '1498', '1499', '1507',
       '1508', '1529', '1538', '1593', 1467, 1500, 1539], dtype=object)

In [105]:
# 지도 시각화의 기준점이 휘발유 가격
# 고로 휘발유 가격을 정제하여 의미있는 데이터만 남김
st_raw_price_exist = st_raw[ st_raw['휘발유'] != '-' ]
st_raw_price_exist.shape

(416, 10)

In [106]:
# 데이터를 정수형으로 형변환 astype() <- 시리즈
st_raw_price_exist.loc[:,'휘발유'] = st_raw_price_exist.loc[:,'휘발유'].astype(int)

In [107]:
st_raw_price_exist['휘발유'].unique()

array([1463, 1473, 1478, 1483, 1489, 1498, 1503, 1509, 1527, 1528, 1569,
       1579, 1585, 1617, 1695, 1459, 1475, 1488, 1495, 1518, 1598, 1482,
       1486, 1487, 1514, 1517, 1538, 1545, 1597, 1650, 1688, 1698, 1458,
       1468, 1492, 1496, 1510, 1513, 1515, 1519, 1558, 1557, 1578, 1658,
       1694, 1434, 1565, 1449, 1457, 1462, 1479, 1485, 1529, 1533, 1535,
       1548, 1550, 1553, 1555, 1568, 1583, 1593, 1633, 1499, 1508, 1540,
       1551, 1699, 1476, 1477, 1497, 1525, 1549, 1589, 1595, 1596, 1493,
       1885, 1466, 1507, 1725, 1445, 1448, 1454, 1506, 1559, 1572, 1443,
       1444, 1556, 1629, 1637, 1739, 1440, 1447, 1467, 1500, 1539],
      dtype=int64)

In [108]:
# 시리즈 타입 확인 
st_raw_price_exist['휘발유'].dtype

dtype('int32')

In [109]:
st_raw_price_exist['휘발유'].min(), st_raw_price_exist['휘발유'].max(), st_raw_price_exist['휘발유'].mean() 

(1434, 1885, 1510.0384615384614)

In [110]:
st_raw_price_exist.head(3)

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,부산광역시,은마석유 노포주유소,부산 금정구 중앙대로 2191 (노포동),GS칼텍스,051-508-1645,Y,-,1463,1308,1050
1,부산광역시,훼미리알뜰주유소,부산 금정구 두실로 2 (남산동),알뜰주유소,051-582-5882,N,-,1473,1318,-
2,부산광역시,청룡명품주유소,부산 금정구 중앙대로 2130,현대오일뱅크,051-508-5150,N,-,1478,1318,-


In [111]:
# 구 이름 추출 
# 빈칸 기준으로 분리(split)
tmp_gu = st_raw_price_exist['주소'].apply( lambda x : x.split()[1] )
st_raw_price_exist['자치구'] = tmp_gu

C:\Users\sun41\AppData\Local\Continuum\anaconda3\envs\analysis\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [112]:
# 부산의 총 자치구 중복성 검사 , 16개 확인 완료
st_raw_price_exist['자치구'].unique(), len(st_raw_price_exist['자치구'].unique())

(array(['금정구', '서구', '수영구', '연제구', '영도구', '중구', '해운대구', '기장군', '남구', '동구',
        '동래구', '부산진구', '북구', '사상구', '사하구', '강서구'], dtype=object), 16)

In [ ]:
# 데이터 베이스 만들기 

In [113]:
st_raw_gasoline = pd.pivot_tabel( st_raw_price_exist, index ='자치구' )

AttributeError: module 'pandas' has no attribute 'pivot_tabel'